In [17]:
import networkx as nx

In [18]:
INPUT_TEST_ = """RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE"""
INPUT_TEST = [list(s) for s in INPUT_TEST_.split('\n')]

INPUT_TEST_1_ = """AAAAAA
AAABBA
AAABBA
ABBAAA
ABBAAA
AAAAAA"""
INPUT_TEST_1 = [list(s) for s in INPUT_TEST_1_.split('\n')]

with open('d12_in.txt', 'r') as f:
    INPUT_ = f.read()
INPUT = [list(s) for s in INPUT_.split('\n')]

In [19]:
def in_bounds(v, map_):
    i, j = v
    return 0 <= i <= len(map_)-1 and 0 <= j <= len(map_[0])-1

def get_nbhs(v, map_):
    i, j = v
    nbhs = [(i, j-1), (i-1, j), (i, j+1), (i+1, j)]
    nbhs = [n for n in nbhs
            if in_bounds(n, map_)
            and map_[i][j] == map_[n[0]][n[1]]]
    return nbhs

In [20]:
def dfs(map_, v, discovered, cc):
    cc.add(v)
    discovered[v] = True
    nbhs = get_nbhs(v, map_)
    for n in nbhs:
        if not discovered[n]:
            dfs(map_, n, discovered, cc)

def connected_components(map_):
    discovered = {(i, j): False
                  for i in range(len(map_))
                  for j in range(len(map_[0]))}
    available_verts = {v for v in discovered if not discovered[v]}

    while available_verts:
        v = list(available_verts)[0]
        cc = set()
        dfs(map_, v, discovered, cc)
        available_verts = available_verts - cc
        yield cc

In [21]:
def compute_cost(cc, map_, p1=True):
    perimeter = 0
    area = len(cc)
    for v in cc:
        if p1:
            perimeter += 4-len(get_nbhs(v, map_))
        else:
            perimeter += len(classify_node(v, map_))
    return perimeter * area

def get_nbhs_and_dirs(v, map_):
    i, j = v
    nbhs = [((i, j-1), 'W'), ((i-1, j), 'N'), ((i, j+1), 'E'), ((i+1, j), 'S')]
    nbhs = [(n, d) for (n, d) in nbhs
            if in_bounds(n, map_)
            and map_[i][j] == map_[n[0]][n[1]]]
    return nbhs

def is_corner(v, map_, type_):
    i, j = v
        
    nd_lst = get_nbhs_and_dirs(v, map_)
    dirs = [nd[1] for nd in nd_lst]
    if type_[0] in dirs and type_[1] in dirs:
        match type_:
            case 'NW':
                w = (i-1, j-1)
                c1 = not in_bounds(w, map_)
                c2 = map_[w[0]][w[1]] != map_[i][j]
                return c1 or c2
            case 'SE':
                w = (i+1, j+1)
                c1 = not in_bounds(w, map_)
                c2 = map_[w[0]][w[1]] != map_[i][j]
                return c1 or c2
            case 'SW':
                w = (i+1, j-1)
                c1 = not in_bounds(w, map_)
                c2 = map_[w[0]][w[1]] != map_[i][j]
                return c1 or c2
            case 'NE':
                w = (i-1, j+1)
                c1 = not in_bounds(w, map_)
                c2 = map_[w[0]][w[1]] != map_[i][j]
                return c1 or c2
    elif type_[0] not in dirs and not type_[1] in dirs:
        return type_ in ['NW', 'NE', 'SW', 'SE']
    return False

def classify_node(v, map_):
    nd = get_nbhs_and_dirs(v, map_)
    dirs_nbhs = [nd_[1] for nd_ in nd]
    lst = []
    if len(nd) == 1:
        n, d = nd[0]
        match d:
            case 'W':
                lst = ['NE', 'SE']
            case 'E':
                lst = ['NW', 'SW']
            case 'N':
                lst = ['SE', 'SW']
            case 'S':
                lst = ['NE', 'NW']
    elif len(nd) == 4:
        for type_ in ['NW', 'NE', 'SW', 'SE']:
            if is_corner(v, map_, type_):
                lst.append(type_)
    elif len(nd) == 3:
        if 'N' not in dirs_nbhs:
            for type_ in ['SW', 'SE']:
                if is_corner(v, map_, type_):
                    lst.append(type_)
        elif 'S' not in dirs_nbhs:
            for type_ in ['NW', 'NE']:
                if is_corner(v, map_, type_):
                    lst.append(type_)
        elif 'W' not in dirs_nbhs:
            for type_ in ['NE', 'SE']:
                if is_corner(v, map_, type_):
                    lst.append(type_)
        elif 'E' not in dirs_nbhs:
            for type_ in ['SW', 'NW']:
                if is_corner(v, map_, type_):
                    lst.append(type_)
    elif len(nd) == 2:
        if 'N' in dirs_nbhs and 'W' in dirs_nbhs: 
            if is_corner(v, map_, 'NW'):
                lst.append('NW')
            if is_corner(v, map_, 'SE'):
                lst.append('SE')
        elif 'N' in dirs_nbhs and 'E' in dirs_nbhs:
            if is_corner(v, map_, 'NE'):
                lst.append('NE')
            if is_corner(v, map_, 'SW'):
                lst.append('SW')
        elif 'S' in dirs_nbhs and 'W' in dirs_nbhs:
            if is_corner(v, map_, 'SW'):
                lst.append('SW')
            if is_corner(v, map_, 'NE'):
                lst.append('NE')
        elif 'S' in dirs_nbhs and 'E' in dirs_nbhs:
            if is_corner(v, map_, 'SE'):
                lst.append('SE')
            if is_corner(v, map_, 'NW'):
                lst.append('NW')
    elif len(nd) == 0:
        lst = ['NW', 'NE', 'SW', 'SE']
    return lst

Part 1

In [22]:
m = INPUT
total = 0
for cc in connected_components(m):
    total += compute_cost(cc, m)
total

1370100

In [23]:
m = INPUT

G = nx.Graph()

for i in range(len(m)):
    for j in range(len(m[0])):
        v = (i, j)
        G.add_node(v)
        for n in get_nbhs(v, m):
            G.add_edge(v, n)

total = 0
for cc in nx.connected_components(G):
    total += compute_cost(cc, m)
total

1370100

Part 2

In [24]:
m = INPUT
conn_comp = connected_components(m)
total = 0
for cc in conn_comp:
    total += compute_cost(cc, m, p1=False)
total

818286

In [25]:
m = INPUT

G = nx.Graph()

for i in range(len(m)):
    for j in range(len(m[0])):
        v = (i, j)
        G.add_node(v)
        for n in get_nbhs(v, m):
            G.add_edge(v, n)

conn_comp = nx.connected_components(G)

total = 0
for cc in conn_comp:
    total += compute_cost(cc, m, p1=False)
total

818286